# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 13/Set até às 23:59.<br />
Grupo: 1 ou 2 pessoas.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO disponibilizar o arquivo com os *access keys/tokens* do Twitter.**


### Check 3: 

Até o dia 06 de Setembro às 23:59, o notebook e o xlsx devem estar no Github com as seguintes evidências: 
    * Conta no twitter criada.
    * Produto escolhido.
    * Arquivo Excel contendo a base de treinamento e teste já classificado.

Sugestão de leitura:<br />
http://docs.tweepy.org/en/v3.5.0/index.html<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Preparando o ambiente

Instalando a biblioteca *tweepy* para realizar a conexão com o Twitter:

In [3]:
%%capture

#Instalando o tweepy e textblob
!pip install tweepy

Importando as Bibliotecas que serão utilizadas. Esteja livre para adicionar outras.

In [4]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

Para realizar a captura dos dados é necessário ter uma conta cadastrada no twitter:

* Conta: ***@sabrina49834572***


1. Caso ainda não tenha uma: https://twitter.com/signup
1. Depois é necessário registrar um app para usar a biblioteca: https://apps.twitter.com/
1. Dentro do registro do App, na aba Keys and Access Tokens, anotar os seguintes campos:
    1. Consumer Key (API Key)
    1. Consumer Secret (API Secret)
1. Mais abaixo, gere um Token e anote também:
    1. Access Token
    1. Access Token Secret
    
1. Preencha os valores no arquivo "auth.pass"

**ATENÇÃO**: Nunca divulgue os dados desse arquivo online (GitHub, etc). Ele contém as chaves necessárias para realizar as operações no twitter de forma automática e portanto é equivalente a ser "hackeado". De posse desses dados, pessoas mal intencionadas podem fazer todas as operações manuais (tweetar, seguir, bloquear/desbloquear, listar os seguidores, etc). Para efeito do projeto, esse arquivo não precisa ser entregue!!!

In [5]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Coletando Dados

Agora vamos coletar os dados. Tenha em mente que dependendo do produto escolhido, não haverá uma quantidade significativa de mensagens, ou ainda poder haver muitos retweets.<br /><br /> 
Configurando:

In [6]:
#Produto escolhido:
produto = 'RedBull'

#Quantidade mínima de mensagens capturadas:
n = 500

#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [7]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang).items():    
    msgs.append(msg.text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [8]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t2])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t2:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
## Classificando as Mensagens

Agora você deve abrir o arquivo Excel com as mensagens capturadas e classificar na Coluna B se a mensagem é relevante ou não.<br /> 
Não se esqueça de colocar um nome para a coluna na célula **B1**.<br /><br />
Fazer o mesmo na planilha de Controle.

___
## Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Escreva o seu código abaixo:

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.[OK]<br />
* Corrigir separação de espaços entre palavras e/ou emojis.[OK]
* Propor outras limpezas/transformações que não afetem a qualidade da informação.



In [9]:
readerTrain = pd.read_excel('{0}.xlsx'.format(produto))
readerTest = pd.read_excel('{0}.xlsx'.format(produto),sheetname = 'Teste')
train = []
test = []

for i in range(0,t):
    train.append([readerTrain['Treinamento'][i], readerTrain['Util'][i]])

for i in range(0,n-t):
    test.append([readerTest['Teste'][i],readerTest['Util'][i]])
    

In [29]:
def multiply(n):
    total = 1
    for i in range(0, len(n)):
        total *= n[i]
    return total

class NaiveBayes:
    def __init__(self):
        self.YesFreq = 0
        self.NoFreq = 0
        self.WordFreqList = [["redbull",0,0]]
                            #[Palavra, FreqNo, FreqYes]
        self.TotalWord = [1,0,0]
                            #Palavra, FreqNo, FreqYes
        
    def train(self,train):
        for tweet in train:
            if tweet[1]: self.YesFreq+=1 #1 = Coluna Util
            else: self.NoFreq+=1
            tweet[0] = self.normalizeTweet(tweet[0])#0 = Coluna Treinamento
            WordList = tweet[0].split(' ') #0 = Coluna Treinamento
            for word in WordList:
                onList = False
                for freq in self.WordFreqList:
                    if word == freq[0]:
                        onList = True
                        if tweet[1]: 
                            freq[2]+=1 #1 = Coluna Util
                            self.TotalWord[2]+=1
                        else: 
                            freq[1]+=1
                            self.TotalWord[1]+=1
                
                if not onList:
                    self.TotalWord[0]+=1
                    if tweet[1]: self.WordFreqList.append([word,0,1]) #1 - Coluna Util
                    else: self.WordFreqList.append([word,1,0])
        return 0
    
    def test(self, test):
        acertoTotal = 0 
        erroTotal = 0
        acertoRelevante = 0
        erroRelevante = 0
        acertoIrrelevante = 0
        erroIrrelevante = 0
        for tweet in test:
            if self.analyze(tweet)==tweet[1]:
                acertoTotal+=1
                if tweet[1]: acertoRelevante+=1
                else: acertoIrrelevante+=1
            else:
                erroTotal+=1
                if tweet[1]: erroRelevante+=1
                else: erroIrrelevante+=1
        print("Positivo Falso: " + str((erroRelevante/(n-t))*100)+"%")
        print("Positivo Verdadeiro: " + str((acertoRelevante/(n-t))*100)+"%")
        print("Negativo Falso: " + str((erroIrrelevante/(n-t))*100)+"%")
        print("Negativo Verdadeiro: " + str((acertoIrrelevante/(n-t))*100)+"%")
        
        print()

        print("Acerto: " + str(((acertoIrrelevante+acertoRelevante)/(n-t))*100)+"%")
        print("Erro: "  + str(((erroIrrelevante+erroRelevante)/(n-t))*100)+"%")

        print()

        print("Total: " + str(((erroIrrelevante+erroRelevante+acertoIrrelevante+acertoRelevante)/(n-t))*100)+"%")
             
    
    def analyze(self,tweet):
        tweet[0] = self.normalizeTweet(tweet[0])#0 = Coluna Treinamento
        WordList = tweet[0].split(' ') #0 = Coluna Treinamento
        relevancia = [] #acho que nao ta sendo usado
        probWord = []
        for word in WordList:
            onList = False
            for freq in self.WordFreqList:
                if word == freq[0]:
                    onList = True
                    probNo = (freq[1]+1)/(self.TotalWord[1]+self.TotalWord[0])
                    probYes = (freq[2]+1)/(self.TotalWord[2]+self.TotalWord[0])
                    probTotal = (freq[1]+freq[2]+1)/self.TotalWord[0]
                    probWord.append([probTotal,probNo,probYes])
                    
            if not onList:
                probWord.append([1/self.TotalWord[0],1/self.TotalWord[1],1/self.TotalWord[2]])
        
        probTweet = (multiply(probWord[:][1]),multiply(probWord[:][2]))
        if((probTweet[0]>probTweet[1])): return 1
        else: return 0
 
    def normalizeTweet(self,tweet):
        tweet.lower()
        tweet = tweet.replace(',',' ')
        tweet = tweet.replace('!',' ')
        tweet = tweet.replace('$',' ')
        tweet = tweet.replace('%',' ')
        tweet = tweet.replace('&',' ')
        tweet = tweet.replace('*',' ')
        tweet = tweet.replace('(',' ')
        tweet = tweet.replace(')',' ')
        tweet = tweet.replace('-',' ')
        tweet = tweet.replace('+',' ')
        tweet = tweet.replace('=',' ')
        tweet = tweet.replace('_',' ')
        tweet = tweet.replace('.',' ')
        tweet = tweet.replace(':',' ')
        tweet = tweet.replace(';',' ')
        tweet = tweet.replace('?',' ')
        tweet = tweet.replace('/',' ')
        tweet = tweet.replace('\n',' ')
        for i in range(0,4): tweet = tweet.replace('  ',' ')
        return tweet
                              
    def show(self):
        print("SIMs: " + str(self.YesFreq))
        print("NÃOs: " + str(self.NoFreq))
        print("TOTAL: " + str(self.YesFreq+self.NoFreq))
    
    def showAll(self):
        print(self.WordFreqList)
        print()
        print("SIMs: " + str(self.YesFreq))
        print("NÃOs: " + str(self.NoFreq))
        print("TOTAL: " + str(self.YesFreq+self.NoFreq))
        


In [30]:
Classificador = NaiveBayes()
Classificador.train(train)
Classificador.test(test)

Positivo Falso: 2.5%
Positivo Verdadeiro: 3.5000000000000004%
Negativo Falso: 12.5%
Negativo Verdadeiro: 81.5%

Acerto: 85.0%
Erro: 15.0%

Total: 100.0%


___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Opcionalmente:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.[  ]

___
## Concluindo

Escreva aqui a sua conclusão.[OK]<br /> 
Faça um comparativo qualitativo sobre as medidas obtidas.[OK]<br />
Explique como são tratadas as mensagens com duplanegação e sarcasmo.[OK]<br />
Proponha um plano de expansão. Por que eles devem continuar financiando o seu projeto?[OK]<br />

Opcionalmente: 
* Discorrer por que não posso alimentar minha base de Treinamento automaticamente usando o próprio classificador, aplicado a novos tweets.[ ]
* Propor diferentes cenários de uso para o classificador Naive-Bayes. Cenários sem intersecção com este projeto.[ ]
* Sugerir e explicar melhorias reais no classificador com indicações concretas de como implementar (não é preciso codificar, mas indicar como fazer e material de pesquisa sobre o assunto).[OK]


O nosso Classificador Naive-Bayes analisa propabilidade de um tweet ser relevante baseado nas palavras usadas nele. Para conhecer a probalidade de relevancia de uma palavra fornecemos um primeiro grupo de tweet para treina-lo.
Tivemos um percentual de analises corretas bem alta, porem o numero de Positivos Verdadeiros foi bastante baixo, isso pode estar ligada a poucos exemplos de Positivos em seu treino (Qualidade do Dataframe de Treino) e já que não existe uma analise completa do tweet, nosso clasificador não reconhece contexto, sarcasmo, duplas negações ou qualquer outro aspecto da frase que não seja a probalidade de suas palavras tornararem o proprio tweet relevante, isso faz com que o classificador cometa diverssos erros, já que podemos usar certas palavras em ambos os casos (Tweets relevantes e Irrelevantes).

___
Em nossas proximas iterações, para podermos aprimorar nosso Classificador e sem alterar o tipo dele (Naive-Bayes), vamos por exemplo:
    
* Ao se deparar com o Tweet: "Eu gosto de beber Redbull sextas a noite", alem de analisar e registrar cada uma das palavras nossa classificador passará a analisar tambem fragmentos do tweet, como: "Eu gosto de", "beber Redbull" e "sextas a noite"<br /> 
* Passar a não registrar artigos de ligação, como: "de", "do", "dos", "da", "das", etc.<br />
* Fornecer ao Classificador um grupo de regras para transformar abreviações em sua versão completa e corrigir erros de escrita, para que dessa forma não registre duas palavras iguais como duas diferentes, tal como: "vc" e "você", aumentando a qualidade do treinamento e de analise do Classificador.